In [1]:
import torch.version
from vit import ViT
import einops
from pathlib import Path
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
from torchvision.transforms import Compose, Resize, ToTensor, Normalize, RandomHorizontalFlip, RandomCrop
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

print(torch.__version__)

2.1.2


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

patch_size = 16
latent_size = 768
n_channels = 3
num_heads = 12
num_encoders = 12
dropout = 0.1
num_classes = 10
size = 224

epochs = 10
base_lr = 10e-3
weight_decay = 0.03
batch_size = 25

cuda:0


In [3]:
# I resize the input data to 224x224, since that is the training resolution used in the paper.
# The mean and std values used to normalize CIFAR10 data is from here: https://github.com/kentaroy47/vision-transformers-cifar10/blob/main/train_cifar10.py
import torch.utils


transform_training_data = Compose(
    [RandomCrop(32, padding=4), Resize((224)), RandomHorizontalFlip(), ToTensor(), Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]
    )

# Load train and test datasets
filepath = Path('.')
train_data = torchvision.datasets.CIFAR10(
    root=filepath, train=True, download=True, transform=transform_training_data)

test_data = torchvision.datasets.CIFAR10(
  root=filepath, train=False, download=False, transform=transform_training_data)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified


In [4]:
def validate_model(model, dataloader, criterion, device):
  model.eval()  # Set model to evaluation mode
  running_loss = 0.0
  correct_predictions = 0
  total_samples = 0

  with torch.no_grad():  # Disable gradient calculation
    for inputs, labels in dataloader:
      inputs, labels = inputs.to(device), labels.to(device)

      outputs = model(inputs)
      loss = criterion(outputs, labels)
      running_loss += loss.item() * inputs.size(0)

      _, predicted = torch.max(outputs, 1)
      correct_predictions += (predicted == labels).sum().item()
      total_samples += labels.size(0)

      # Clear cache to free up memory
      torch.cuda.empty_cache()

  avg_loss = running_loss / total_samples
  accuracy = correct_predictions / total_samples
  return avg_loss, accuracy

In [5]:
my_model = ViT(num_encoders, latent_size, device, num_classes).to(device)

# Betas used for Adam in paper are 0.9 and 0.999, which are the default in PyTorch
optimizer = optim.Adam(my_model.parameters(), lr=base_lr, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.LinearLR(optimizer)

TypeError: Parameter.__new__() takes from 1 to 3 positional arguments but 4 were given

In [18]:
def main():
    my_model.train().to(device)

    optimizer = optim.Adam(my_model.parameters(), lr=base_lr, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.LinearLR(optimizer)

    for epoch in tqdm(range(epochs), total=epochs):
        running_loss = 0.0
        for batch_idx, (inputs, targets) in enumerate(tqdm(trainloader)):

            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()

            outputs = my_model(inputs)

            loss = criterion(outputs, targets)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            #Visualizer for fit

            if batch_idx % 200 == 0:
                print('Batch {} epoch {} has loss = {}'.format(batch_idx, epoch, running_loss/200))
                running_loss = 0

        scheduler.step()
        torch.cuda.empty_cache()
    torch.cuda.empty_cache()

    torch.save(my_model.state_dict(), 'my_model3.pth')


    test_accuracy = validate_model(my_model, testloader, criterion, device)
    print(f'Test Accuracy: {test_accuracy}%')


In [24]:
if __name__ == "__main__":  
    main()

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 1 but got size 25 for tensor number 1 in the list.